In [15]:
import os
import polars as pl
import altair as alt

import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [4]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [5]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [6]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [147]:
barvy = pl.concat(
        [
            vyvoj_vyskytu(r'[^\w]rud[áýáíěoaé]\w{0,3}','rudá'),
            vyvoj_vyskytu(r'[^\w]červen[áýáíěoaé]\w{0,3}','červená'),
            vyvoj_vyskytu(r'[^\w]zelen[áýíěoaé]\w{0,3}','zelená'),
            vyvoj_vyskytu(r'[^\w]hněd[áýíěoaé]\w{0,3}','hnědá'),
            vyvoj_vyskytu(r'[^\w]čern[áýíěoaé]\w{0,3}','černá'),
            vyvoj_vyskytu(r'[^\w]bíl[áýíěoaé]\w{0,3}','bílá'),
            vyvoj_vyskytu(r'[^\w]šed[áýíěoaé]\w{0,3}','šedá'),
            # vyvoj_vyskytu(r'(?i)[^\w]růžov','růžová')
        ]
)

In [149]:
poradi=['rudá','červená','zelená','černá','bílá','hnědá','růžová','šedá']

In [159]:
barvy_graf = alt.Chart(
    barvy.to_pandas(),
    width=300,
    height=50,
    title=alt.Title(
        'Barvy zmiňované v poslaneckých projevech',
        subtitle=['Pro vyloučení jmen a názvů (Josef Černý, Červená Voda…) hledáme','pouze zmínky malými písmeny.']
    )
).mark_bar(
    width=2
).encode(
    alt.X("datum:T", title=None),
    alt.Y("pomer:Q", axis=alt.Axis(title=None, orient="right",format="%")),
    alt.Row(
        "co:N", title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'),
        sort=poradi
    ),
    alt.Color("co:N", title=None, legend=None, scale=alt.Scale(
        range=['#D6534B','#D6534B','#70871E','#292829','#445B78','#5E2D3A','#9eb8c1','#E09DA3']
    ), sort=poradi
             )
).resolve_axis(
    x="independent"
).configure_view(
    stroke='transparent'
)

barvy_graf

alt.Chart(...)

In [157]:
barvy_graf.save("grafy/barvy_graf.png")